In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import duckdb


In [2]:
# function for extracting the profile link of each organizer
def get_plinks(results):
    a_tags = results.find_all('a')
    profile_url = []
    for i in a_tags:
        end_url = i['href']
        link = 'https://ocimpact.app.swapcard.com'+end_url
        profile_url.append(link)
    return profile_url

In [4]:
driver = webdriver.Chrome()

In [6]:

driver.get('https://ocimpact.app.swapcard.com/widget/event/oc2023/people/RXZlbnRWaWV3XzQ1NTQwOA==?showActions=true')
time.sleep(2)

In [7]:
# scroll down to the bottom of the page
while True:
    driver.execute_script('window.scrollBy(0, 1500)')
    time.sleep(2)
    # check if at bottom of page
    if driver.execute_script('return window.innerHeight + window.pageYOffset >= document.body.offsetHeight'):
        break

# parsing the data using BeautifulSoup
content = driver.page_source
soup = BeautifulSoup(content,"html.parser")
results=soup.find(class_='infinite-scroll-component')

# scrap the url of deligates profile
profile_links = get_plinks(results=results)

#clean the urls by removing the duplicates
profile_links = [url for url in profile_links if not url.endswith('openConnection=true')]



In [9]:
# declaring required lists
names = []
job_title = []
organization = []
qa_values = []
quest1, quest2, quest3, quest4, quest5, quest6, quest7, quest8, quest9 = [],[],[],[],[],[],[],[],[]

questions = ['What are you working on in the field of social and economic justice?','What do you need from the Delegate community to move this work forward?',
            'What are you able and willing to contribute to the Delegate community?',"I'm comfortable conversing in:",
            "What are you able and willing to contribute to the Delegate community?","Sustainable Development Goals (SDGs)","Ribbons","Geographic Focus","Tags"]


In [10]:
# maximum waiting time to load a webpage is set to 30 sec
parser = []
max_wait_time = 30
for i in profile_links:
    driver.get(i)
    time.sleep(2)
    try:
        element_present = EC.presence_of_element_located((By.TAG_NAME, 'body'))
        WebDriverWait(driver, max_wait_time).until(element_present)
        print("Website is ready!")
        content = driver.page_source
        soup = BeautifulSoup(content,"html.parser")
        result = soup.find(class_ = 'style__UIBox-ui__sc-134aw2g-0 gMgeSq details__Box-ea__sc-pa7wq-0 iUiEoC')
        parser.append(result)
    except TimeoutException:
        print(f'couldnt load website {i}')
driver.quit()




Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!
Website is ready!


In [11]:
for i in parser:
    try:     
        name=i.find('h2',class_='style__Name-cmp__sc-1s7e137-1 jhjTCw')
        name=name.text.strip()
        names.append(name)
    except:
        names.append('')

    try:
        job = i.find('h4','style__Job-cmp__sc-1s7e137-2 dzeTcv')
        job = job.text.strip()
        job_title.append(job)
    except:
        job_title.append('')

    try:
        org = i.find('h3','style__Organization-cmp__sc-1s7e137-3 cVzOUy')
        org = org.text.strip()
        organization.append(org)
    except:
        organization.append('')
    
    qa = i.find_all(class_='style__Wrapper-cmp__sc-165xmjy-0 kxlYqL')
    qa_values = []

    for elem in qa:
        quest = elem.find(class_='style__Name-cmp__sc-165xmjy-1 dVHQNt')
        quest = quest.text.strip()
        ans = elem.find(class_='style__Content-cmp__sc-165xmjy-2 dNVyzo')
        ans = ans.text.strip()
        ans = ans.replace('See more','')
        qa_dict ={
            "question" : quest,
            "answer" : ans
        }
        qa_values.append(qa_dict)


        
    for i in qa_values:
        q = i.get('question').strip()
        
        for n in range (9):
            if q == questions[n]:
                globals()[f'quest{n+1}'].append(i.get('answer'))


    curr_questions = [item.get('question').strip() for item in qa_values]

    for j in range(9): 


        if questions[j] not in curr_questions:
            globals()[f'quest{j+1}'].append('')

In [13]:
final = {
    "Name" : names,
    "Job Title" : job_title,
    "Organization" : organization,
    questions[0] : quest1,
    questions[1] : quest2,
    questions[2] : quest3,
    questions[3] : quest4,
    questions[4] : quest5,
    questions[5] : quest6,
    questions[6] : quest7,
    questions[7] : quest8,
    questions[8] : quest9
}
df = pd.DataFrame(final)
df

                                Name  \
0            Rotimi Olawale (he/him)   
1                 Nik Kafka (he/him)   
2               Martin Burt (he/him)   
3          Maryam Montague (she/her)   
4      Ingrid Karangwayire (she/her)   
5               Taio Gardey (he/him)   
6            Willie Mpasuka (he/him)   
7               Grace Moyd (she/her)   
8      Karla Corichi (No preference)   
9             Lashon Amado (she/her)   
10           Liz Fischelis (she/her)   
11                  Pamela Nathenson   
12           Chad Bernstein (he/him)   
13      Christopher Turillo (he/him)   
14  Moiyattu Banya-Keister (she/her)   
15            Kaley Howard (she/her)   
16       Michelle Donnelly (she/her)   
17               Elio Gomez (he/him)   
18                      Caleb Alcime   
19                 J Howard (he/him)   
20            Hailey Barth (she/her)   
21            Gregg Cannady (he/him)   
22          Jasmine Anouna (she/her)   
23  Odomitchi Anikpo (no preference)   


In [67]:
#connecting with duckdb
con = duckdb.connect()
con.register("my_table",df)

result = con.execute("SELECT * FROM my_table ").fetchall()
result

[('Rotimi Olawale (he/him)',
  'Executive Director',
  'Youthhubafrica',
  'I am working on providing educational access and opportunities for under-served girls and women, as well as strengthening capacity of young people for meaningful and gainful employment',
  "I'll need technical advise on having further clarity on sustainability options, looking for connections and opportunities to collaborate with organisations/granmakers doing work in Nigeria/Africa",
  'I have extensive experience working in Nigeria/Africa and with institutions like the African Union. I have a deep understanding of the youth ecosystem as well as youth policy in Africa and I am widely travelled on the African continent, I am willing to contribute my expertise and provide technical advise and support',
  'English',
  '',
  '',
  '',
  ''),
 ('Nik Kafka (he/him)',
  'CEO & Founder',
  'Teach A Man To Fish',
  'Relevant real-world education. Setting young people up to take charge of their lives. Building essential

In [66]:
df.to_csv('data.csv')